# Safely Store ENV variables for Gemini, Chroma DB connection, and vector embeddings
### Additional techniques: cleared memory of any existing variables to avoid accidental exposure. Performed due to significant security risk of exposing PII and PHI for healthcare data.
---

In [96]:
# INSTALLS
#! pip install rank-bm25
#! pip install langchain-classic rank_bm25 langchain-community
#! pip install -U langchain-google-vertexai

In [1]:
# imports 
# # load dotenv + imports for retriever tool
import os
from dotenv import load_dotenv
from langchain_chroma import Chroma 
from langchain.tools import tool
import langchainhub as hub
import chromadb
from langchain_google_genai import GoogleGenerativeAIEmbeddings #type:ignore
import vertexai
from vertexai.language_models import TextEmbeddingModel
# from langchain_community.retrievers import BM25Retriever
# from langchain_core.runnables import RunnableLambda, RunnableSequence, RunnableMap, RunnablePassthrough, RunnableParallel
from langchain_core.runnables import Runnable
# from llama_index.vector_stores.chroma import ChromaVectorStore
# from llama_index.core import StorageContext, VectorStoreIndex, Document
# from llama_index.retrievers.bm25 import BM25Retriever
# from llama_index.core.retrievers import QueryFusionRetriever


/Users/briannamitchell/miniconda3/envs/rag_stable312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Disabling PyTorch because PyTorch >= 2.4 is required but found 2.2.2
PyTorch was not found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:

import os 
# Load environment variables from .env file first
load_dotenv()

#api_key = os.getenv("CHROMA_API_KEY") or getpass("Paste CHROMA_API_KEY: ")
#tenant = os.getenv("CHROMA_TENANT") or input("Enter CHROMA_TENANT: ")
#database = os.getenv("CHROMA_DATABASE") or input("Enter CHROMA_DATABASE: ")
#gemini_key = os.getenv("GEMINI_API_KEY") or getpass("Enter GEMINI_API_KEY: ")
#vertex_ai_key = os.getenv("GOOGLE_APPLICATION_CREDENTIALS") or getpass("Enter GOOGLE_APPLICATION_CREDENTIALS: ")
#openAI_api_key = os.getenv("OPENAI_API_KEY") or getpass("Enter OPENAI_API_KEY")

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY", "")
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = os.getenv("GOOGLE_APPLICATION_CREDENTIALS", "")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "")

# Load environment variables from chromaDB with persistence locally, and prevent unecessary calls to API--more cost-effective approach
persistent_client = chromadb.PersistentClient(path="./chroma_db")

# Ensure the GEMINI_API_KEY environment variable is set
gemini_api_key = os.getenv("GEMINI_API_KEY")
if not gemini_api_key:
	from getpass import getpass
	gemini_api_key = getpass("Enter GEMINI_API_KEY: ")
	os.environ["GEMINI_API_KEY"] = gemini_api_key

# Importing Gemini for embedding
embeddings = GoogleGenerativeAIEmbeddings(model="gemini-embedding-001", api_key=gemini_api_key)


# **Challenges + Mitigation strategies for Performing Vector Search:**
---
#### **Tradeoffs:**
- Originally, my plan was to use the same framework for both retrievers, then fetch the data from my vector store in chroma. However, I found that my BM25 retriever simply would not return any data for either framework--langchain or llamaindex. Later on, I found out that due to how the BM25-Retriever architecture is built, the keyword search library needs access to pull data from **raw text nodes**. This does not align with my current Chroma vector store DB architecture, which is built **with stored embeddings--not original text**. Hence, why orginally no data was being returned in my prior iteration. 

#### **How I solved for this/Why I chose this method?:**
To mitigate for this, I decided to change my framework and use llamaindex for BM25/lexical retriever; it provides a much more seamless integration with the Chroma vector store architecture. Then, I used llamaindex to pass my list of nodes directly to the BM25 retriever, which solved the short-term problem.

#### **Improvement with Llama_Index:**
- Use raw text nodes for BM25 retrieval to ensure compatibility with the keyword search library
- Implement response enhancement by providing additional context or *page_content* for my document queries.

---

## **Pitfall/Tradeoffs with my original approach:**
While attempting to test the combination of langchain and llamaIndex for different retrievers, I came across multiple problems. The main issue, my database and embeddings are in ChromaDB. The issue arises when performing a hybrid retrieval--where you **combine both a semantic, dense retriever and a lexical, keyword precise retriever**. The **root problem stemmed from BM25's architecture**. Semantic retriever seamlessy integrates with my dB because it uses the **vector_store from Chroma**. My BM25 does not integrate well because it **must pass the queried Document objects as nodes**--**something that can only be performed with VectorStoreIndex in LlamaIndex**. Being that Chroma is very langchain dependent and VectorStoreIndex is LlamaIndex dependant, combining these two very different methods would result in uneccessary overhead. I also had to factor in the limited constraint of having to use Gemini 1.5 with VertexAI embeddings, which is a closed-solution.

- If I had to start over, to successfully integrate a hybrid retrieval RAG, I would have used either **OpenAI embeddings, ChatGPT4o, and ChromaDB** to still keep the closed-source solution in production. 

- If building proof of concept locally, I would use **FAISS for my vector store** + replace my **model with Gemma from HuggingFace.** for a more open source solution.

---


# **Final RAG Approach:**
---

## My main consideration...**How can I pivot without losing retrieval quality?****
My plan is to move to a more powerful approach by performing accurate similarity search with my **vector_store in ChromaDb** as my retriever in langchain. This is done to obtain exact matches with the potential of ingesting large documents via **Approximate Nearest Neighbors (ANN) algorithm built-in to ChromaDB**. Then, I will apply **Maximum Marginal Relevance(MMR)**. It's a great pivot from my prior solution. I'm able to optimize for balance between **relevance and diversity in a faster time complexity**. In a live healthcare environment, my agent would come in contact with multiple provider oppinions. In order to accurately generate responses that mimic live production, I would need to compensate for scaling this factor later on. Hence, whyn I chose this method.
**Benefits:**
- Prioritizes relevance and diversity during retrieval
- Compensates for random spikes during data ingestion
- Balances exploration with exploitation
- Ensures my retrieved documents are distinct from each other

# **Creating VectorStore for LangChain + Enhancing Responses by Verifying Context Retrieved: How I Improved Generator Output:** 
---
### Used response enhancement as a guideline for the model to follow when verifying retrieved content, building a better **knowledge base engine** for my agent to fetch. This system design technique results in a more enhanced **RAG performance + quality**. 




In [3]:
# adding response enhancement for generated outputs
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma

#get collection name from chromaDB
#COLLECTION_NAME = "ng12"
#chroma_collection = client.get_collection(name=COLLECTION_NAME)
#embed_model = embeddings

#vectore store
vectorstore = Chroma(
    collection_name="ng12",
    embedding_function=embeddings,
    client=persistent_client
)

#storage context
#storage_context = StorageContext.from_defaults(vectorstore=vectorstore)
#storage_context.persist()

documents = [
    Document(page_content="Shortness of breath with cough or fatigue or chest pain or weight loss or appetite loss (unexplained), 40 and over: possible cancer Lung or mesothelioma", metadata={"referral": "Urgent", "source": "Suspected cancer: recognition and referral (NG12) 2026", "page": "52"}),
    Document(page_content="Bleeding, bruising or petechiae, unexplained: possible cancer Leukaemia", metadata={"referral": "Very urgent", "source": "Suspected cancer: recognition and referral (NG12) 2026", "page": "43"}),
    Document(page_content="Fracture unexplained, 60 and over: possible cancer Myeloma", metadata={"referral": "Unexplained", "source": "Suspected cancer: recognition and referral (NG12) 2026", "page": "55"}),
    Document(page_content="Refer people using a suspected cancer pathway referral for oesophageal cancer if they: have dysphagia or, are aged 55 and over, with weight loss, and they have any of the following: upper abdominal pain, reflux, dyspepsia. [2015, amended 2025]", metadata={"referral": "Suspected cancer pathway referral", "source": "Suspected cancer: recognition and referral (NG12) 2026", "page": "11"}),
    Document(page_content="Skin lesion that raises the suspicion of a basal cell carcinoma: possible cancer Basal cell carcinoma  ", metadata={"referral": "Raises the suspicion of", "source": "Suspected cancer: recognition and referral (NG12) 2026", "page": "58"}),
    Document(page_content="Urinary urgency or frequency, increased and persistent or frequent, particularly more than 12 times per month in women, especially if 50 and over: possible cancer Ovarian", metadata={"referral": "Persistent", "source": "Suspected cancer: recognition and referral (NG12) 2026", "page": "60"}),
    Document(page_content="Upper abdominal pain with low haemoglobin levels or raised platelet count or nausea or vomiting, 55 and over: possible cancer Oesophageal or stomach ", metadata={"referral": "Non-urgent", "source": "Suspected cancer: recognition and referral (NG12) 2026", "page": "40"}),
    Document(page_content="Petechiae unexplained in children and young people: possible cancer Leukaemia", metadata={"referral": "Immediate", "source": "Suspected cancer: recognition and referral (NG12) 2026", "page": "74"})
]

/var/folders/36/76yfxjrs77l1ss7r47fqfykr0000gn/T/ipykernel_1330/1904139494.py:11: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


##### Maximum Marginal Relevance:
---
I use my lambda (ƛ) value as a threshold:
- 0 = Max diversity ƛ=0
- 1 = max relevance ƛ=1

**Potential Tuning for MMR:**
- Navigational/Exact--> ƛ= 0.7 - 0.9: "Is this patient at 403 an urgent referral?"
- Balanced/Research--> ƛ= 0.5 - 0.7: "What is the best referral recommendation based on symptoms of patient403?"
- Exploratory/Diverse--> ƛ= 0.3 - 0.5: "Give me a summary of findings and determine referral status.."

In [4]:
#create retriever from vector store and then apply MMR 

retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={
        "k": 8,
        "fetch_k": 50,
        "lambda_mult": 0.6
    }
)

# Techniques used(for my readme):
- vector store as retrieval
- response enhancement
- MMR to diversify the output response 
- Source attribution--> helps prevent hallucinations in the model to output where information comes from
- add Maximal marginal relevance to ensure a balance of diversity and relevance retrieved

# **Perform Context Engineering**
---
### Tradeoffs:

**Why did I choose this approach for context engineering?**

**Reasoning:** 
At first, my goal was to return a single string of relevant documents. However, I realized that my agent would have trouble parsing my actual data in production. Based on the way the data is formatted in retrieval, I had to make some adjustments, and go a bit deeper into context engineering. Also, one of the key constraints in building this agent included a task to cite the specific sources within the relevant data retrieved from the NG12 documents. I could have kept my single string, however, there would have been significant fine-tuning and overhead later. Performing context engineering was the only plausible to way achieve this goal. 

#### Adding Clinical Context Tool
---

# **STOPPING POINT BEFORE ADDING SOURCE ATTRIBUTION TO GET BETTER CONTEXT:**

# Source attribution Prompt
---

In [31]:
# attrbution
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

attribution_prompt = ChatPromptTemplate.from_template(
    """ You are a precise and accurate clinical AI assistant that provides expert knowledge in
    clinical decision-making support for those who encounter direct patient care.
    You have inherited a persona profile module as your agentic architecture.
    Main Task: Your main task is to determine whether the presented patient requires an urgent referral or not.
    Use the information provided from the text corpus below:
    The National Institute for Health and Care Excellence (NICE) Guideline for Suspected cancer: recognition and referral NICE guideline.
    Once, main task is complete and accurate, you must provide a recommendation that decides post-referral instructions corresponding to the most relevant medical imaging practices.
    If patient does not meet urgent referral criteria, do not recommend any medical imaging practices.

    Answer the following question based ONLY on the provided sources. 
    For each fact or claim in your answer include a citation that refers to the source.

    Do not make up information or provide personal opinions in your responses without verifying answers with evidence.
    You must cite the specific sources you found from the NICE guidelines in this specific format below:
    This is how you are expected to format: [referral type: insert referral, source: name of text corpus, (year published), page: insert page number]
    
    Your source attributes at the end of your responses will look like this template below in practice:
    [referral: Persistent, source: Suspected cancer: recognition and referral (NG12), 2026, page: 60]

    How your input and output will be formatted with citation sources used:
    Question: {question}

    Sources: {sources}

    Your answer: {answer}
    """
)

### Helper functions to further format the sources with citation numbers and generate attributed responses
---

In [32]:
#source formatted strings from documents
def format_sources_with_citations(documents):
    formatted_sources = []
    for i, doc in enumerate(documents, 1):
        source_info = f"[{i} {doc.metadata.get('source', 'Unknown source')}]"
        if doc.metadata.get('page'):
            source_info += f", page {doc.metadata['page']}"
        formatted_sources.append(f"{source_info}\n{doc.page_content}")
    return "\n\n".join(formatted_sources)
    

### Building RAG chain with my source attribution 
---

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
# invoking my retriever for my relevant docs
client=vertexai.Client(vertexai=True, project="CancerRiskAgent", location="us-east4")

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.0,
    client=client,
)

def generated_attributed_response(question: str):
    retrieved_docs = retriever.invoke(question)
    sources_formatted = format_sources_with_citations(retrieved_docs)

    #create attribution chain, I'll use LCEL methods
    attribution_chain = (
        attribution_prompt
        | llm
        | StrOutputParser()
    )
    response = attribution_chain.invoke({
        "question": question,
        "sources": sources_formatted
    })

    return response

Unexpected argument 'vertex_ai_location' provided to ChatGoogleGenerativeAI. Did you mean: 'vertexai'?
/Users/briannamitchell/miniconda3/envs/rag_stable312/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3701: UserWarning: WARNING! vertex_ai_location is not default parameter.
                vertex_ai_location was transferred to model_kwargs.
                Please confirm that vertex_ai_location is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


ValidationError: 1 validation error for ChatGoogleGenerativeAI
client
  Input should be an instance of Client [type=is_instance_of, input_value=<vertexai._genai.client.C...t object at 0x1208e4140>, input_type=Client]
    For further information visit https://errors.pydantic.dev/2.12/v/is_instance_of

In [36]:
question = "What are some symptoms that require an urgent referral?"
answer = generated_attributed_response(question)

ValidationError: 1 validation error for ChatGoogleGenerativeAI
model
  Input should be a valid string [type=string_type, input_value=ChatGoogleGenerativeAI(pr...ata=(), model_kwargs={}), input_type=ChatGoogleGenerativeAI]
    For further information visit https://errors.pydantic.dev/2.12/v/string_type

### Consistency Checking for Accuracy
---